In [1]:
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
all_amz_go = pd.read_csv('F:\\mailae_g\\ALL_MAG_2021+2022.csv', sep=';' )
all_cus_go = pd.read_csv('F:\\mailae_g\\ALL_PL_2022.csv', sep=';')

In [4]:
all_amz_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,16477,Frage von Amazon-Kunde Anja Klose,Skraut Home- Glanzlack Wohnwand. Von dem großen hängeschrank der Boden ist beschädigt. Beim Dübel loch ist was abgebrochen.,Email,freshdeskuser_18653026@channelreply.email,13 09 2021 15:58,26,18,"lukfurniture_de,cr,lukfurniture,rynek_de"
1,83046,Frage von Amazon-Kunde Furkan Aydin(Bestellung: 306-1709843-3097134),Hallo Die schiene von der schublade ist kaputt da fallen die kleinen Kugeln raus. Ich möchte gerne 2 schienen.,Email,freshdeskuser_20098643@channelreply.email,7 12 2021 22:56,12,6,"lukfurniture_de,cr,lukfurniture,rynek_de"
2,92323,Return policy enquiry from Amazon customer Mrs Susan Tolley(Order: 204-9663987-5997940),Hi I purchased the welder back in April 2021 but due to ill health I have only been able to try the welder today. Unfortunately It does not work. I think there maybe a problem with the MIG output to the switch on the gun. I have used a test meter on the contacts and have no output. Please can you advise. Thankyou.,Email,freshdeskuser_20274812@channelreply.email,16 12 2021 13:51,65,40,"cr,rynek_uk,allweld,allweld_uk"
3,165940,Frage von Amazon-Kunde Fjolla(Bestellung: 306-7063158-3705902),Sehr Geehrte Damen und Herren Es geht um meine Bestellung die ich seit Wochen gemacht habe.Auf meine Bestellung steht dass mein Paket möglicherweise verloren gegangen ist.Ich habe dafür Geld bezahlt.Wie können sie mir weiter helfen? Mit Freundliche Grüßen,Email,freshdeskuser_21788405@channelreply.email,19 03 2022 17:47,14,9,"lukfurniture_de,cr,lukfurniture,rynek_de"
4,192109,"? Sie haben einen neuen Preisvorschlag: EUR 60,00 für NeoNail Nagelstudio ... (373940992894)","Preisvorschlag: EUR 60,00 Preisvorschlag: EUR 60,00 Jetzt antworten: https://www.ebay.de/ulk/showOffers/373940992894?boolp=1&amp;mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44861&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Artikel ansehen: https://www.ebay.de/itm/373940992894?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44862&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Unter Angebotspreis EUR 4,89 Dieses Angebot sichern 9 Beobachter meliba601: https://www.ebay.de/ulk/usr/meliba601?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44863&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=4...",Email,freshdeskuser_22171370@channelreply.email,24 04 2022 18:56,1,12,"ServiceMessage,cr,gashio,gashio_de_eb,ebay,rynek_de_eb"


In [5]:
all_cus_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,111345,[DE] [PYTANIE] B07R42W8J9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,shop@folnet.de,10 01 2022 10:47,4,4,"Partner_wysłane,rynek_de"
1,128291,[DE] 302-0014409-7188317 NIEKOMPLETNE,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,info.palulli@gmail.com,30 01 2022 09:48,3,2,"Partner_wysłane,rynek_de"
2,128325,[REKLAMACJA] 205-9594094-3558723 UK,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,polskaksiegarniainternetowa@gmail.com,30 01 2022 10:38,2,2,"Partner_wysłane,rynek_uk,Przypomnienie"
3,131092,[DE] [ZAPYTANIE] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,kotarbauzapytania@gmail.com,2 02 2022 16:10,4,4,"Partner_wysłane,rynek_de,Przypomnienie"
4,133983,[IT] 408-8392161-3066735 ZWROT,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,kotarbauzapytania@gmail.com,7 02 2022 08:58,9,6,"Partner_wysłane,rynek_it,Przypomnienie1,Przypomnienie2"


In [6]:
all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [7]:
# Funkcja do dzielenia komórek na listę, pierwszy argument tekst, drugi argument po czym tekst ma być dzielony 

def split_tags(tags: str, split_char: str) -> list:
    try:
        tags = tags.split(f'{split_char}')
        return tags 
    except Exception:
        return (list[tags])
    

In [8]:
# funkcja do wyciągania z listy, jakiego rynku dotyczą wiadomości

def get_market(lst: list) -> str:
    for i in lst:
        if "rynek" in i:
            return i

In [9]:
# funkcja do wyciągania numerów zamówień z tekstu i tytułu maila. Przyjmuje dwa różne ciągi tekstowe, sprawdza czy jest w tytule jest regex, jeśli nie to sprawdza czy jest w opisie, jeśli nie to zwraca pustą listę

se = re.compile(r"\d{3}-\d{7}-\d{7}")

def order_number(subject: str, description: str) -> list:
    subject = str(subject)
    description = str(description)
    if se.findall(subject) == []:
        return se.findall(description)
    return se.findall(subject)


In [10]:
# funkcja do tworzenia listy unikalnych wartości z kolumn. Dajemy wartość z kolumny i listę. Sprawdza czy wartość już znajduje się na liście, jeśli nie to ją dodaje

def all_order(order: list, new_list: list) -> list:
    try:
        if order[0] in new_list:
            return new_list
        else:
            new_list.append(order)
            return new_list
    except Exception:
        i =1

In [11]:
# druga funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

on = re.compile(r"\d{3}-\d{7}-\d{7}")
cc = re.compile(r"w{2}")
cc2 = re.compile(r'\[\w{2}\]')

def clear_title(title_list: list) -> list:
    for el in title_list:
        print(el)
    list_to_remove = []
    for i in range(len(title_list)):
        if re.match(on, title_list[i]):
            title_list.remove(i)
        elif re.match(cc, title_list[i]):
            title_list.remove(i)
        elif re.match(cc2, title_list[i]):
            title_list.remove(i)
    return title_list

In [12]:
# trzecia funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def num_there(s):
    return any(i.isdigit() for i in s)

def find_words_with_more_than_four_characters(words_with_tags):
  words = []
  for i in words_with_tags:
    if num_there(i):
      continue
    w = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', i)
    w = w.strip(" ")
    if len(w) >= 4:
      words.append(w)

  return words

In [13]:
# czwarta funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def find_words_with_more_than_four_characters2(subject: str) -> list:
  words = []
  new_subject = ''
  for l in subject:
    l = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', l)
    new_subject += l
  words_temp = new_subject.split(' ')
  for i in words_temp:
    if len(i) >= 4:
      words.append(i)

  return words

In [14]:
# Funkcja do zliczania elementów w liscie, zwraca listę unikatowych wartości i listę unikatowych wartości z informacją ile razy występowały

from collections import Counter

def count_occurrences(input_list):
    unique_values = set(input_list)
    occurrence_count = Counter(input_list)

    return list(unique_values), occurrence_count

In [15]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [16]:
# Zmiana tytułów na lowercase

all_amz_go['Subject'] = [str(row).lower() for row in all_amz_go['Subject']]
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]

In [17]:
# przekształcenie stringów w komórkach kolumny tags na listę, dzielone po przecinku 

all_amz_go['Tags'] = [split_tags(row, ',')for row in all_amz_go['Tags']]
all_cus_go['Tags'] = [split_tags(row, ',')for row in all_cus_go['Tags']]

In [18]:
# utworzenie kolumny, w której jest napisane jakiego rynku dotyczy wiadomość, na podstawie kolumny tags

all_amz_go['rynek'] = [get_market(row) for row in all_amz_go['Tags']]
all_cus_go['rynek'] = [get_market(row) for row in all_cus_go['Tags']]

In [19]:
# usunięcie wierszy, które w kolumnie 'rynek' nie maję 'rynek_de'

# all_amz_go.drop(all_amz_go[all_amz_go['rynek'] != 'rynek_de'].index,  inplace=True)
# all_cus_go.drop(all_cus_go[all_cus_go['rynek'] != 'rynek_de'].index,  inplace=True)

In [20]:
# wyciągnięcie numeru zamówień z kolumn subject i description

all_amz_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_amz_go['Subject'], all_amz_go['Description'])]
all_cus_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]

In [21]:
# resetowanie indexów

all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [22]:
# zmiana obiektów z kolummny 'Created Time' na daty

from datetime import datetime

format_date = "%d %m %Y %H:%M"

all_amz_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_amz_go['Created Time']] 
all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 

In [23]:
# wykorzystanie funkcji do splitowania aby utworzyć kolumnę 

all_amz_go['title_list'] = [split_tags(row, ' ') for row in all_amz_go['Subject']]
all_cus_go['title_list'] = [split_tags(row, ' ') for row in all_cus_go['Subject']]

In [24]:
# sprawdzenie czy funcja split_tags zadzaiała poprawnie

all_cus_go['title_list'][1]

['[de]', '302-0014409-7188317', 'niekompletne']

W tym momencie postanowieł sprawdzić, czy zamówienia, z jednej i drugiej tabeli się pokrywają w jakimś najmniejszym stopniu

In [25]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy jednego data set

all_amz_go_list = []
[all_order(row, all_amz_go_list) for row in all_amz_go['Order_number']]
all_amz_go_list = [item for sublist in all_amz_go_list for item in sublist]

In [26]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy drugiego data set

all_cus_go_list = []
[all_order(row, all_cus_go_list) for row in all_cus_go['Order_number']]
all_cus_go_list = [item for sublist in all_cus_go_list for item in sublist]

In [27]:
# zamieniam stworzyonych list na set

mcg_order_set = set(all_amz_go_list)
mag_rynek_de_order_set = set(all_cus_go_list)

In [28]:
# dodanie kontrolnie jednej takiej samej wartości do obu setów

mcg_order_set.add('123')
mag_rynek_de_order_set.add('123')

In [29]:
# porównanie dwóch

y = mcg_order_set.intersection(mag_rynek_de_order_set)

In [30]:
# Wyświetlenie ile mamy takich samych numerów zamówień w jednym i drugim secie

len(y)

11037

Powrót do preprocesingu danych

In [31]:
# weryfickaja działania funkcji do czyszczenia tytułów maili

print(find_words_with_more_than_four_characters2(all_cus_go['Subject'][5552]))

['zapytanie', 'amandine', 'perez']


In [32]:
# utworzenie nowej kolumny label i dodanie do niej słów, które nadają się na labelki

all_cus_go['label'] = [find_words_with_more_than_four_characters2(row) for row in all_cus_go['Subject']]

In [33]:
# sprawdzenie jak wygląda labels dla 10 przykładowych wierszy

all_cus_go['label'].head(10)

0                    [pytanie]
1               [niekompletne]
2                 [reklamacja]
3                  [zapytanie]
4                      [zwrot]
5          [zwrot, reklamacja]
6                 [reklamacja]
7                 [reklamacja]
8    [zapytanie, asin, bfwdtm]
9                    [pytanie]
Name: label, dtype: object

In [34]:
# Stworzenie listy ze wszystkimi wartościami z kolumny labels

label_list = []
[all_order(row, label_list) for row in all_cus_go['label']]
label_list = [item for sublist in label_list for item in sublist]

In [35]:
# Stworzenie dwóch list na podstawie listy wartości z kolumny['label'], jedna lista to unikatowe wartości, druga lista unikatowe wartości z informacją ile dana wartość występowała

uniq_vales, uniq_values_count = count_occurrences(label_list)

In [36]:
# Stworzenie obiektu spacy z polskim słownikiem

nlp = spacy.load("pl_core_news_md")

In [37]:
# Zmiana unikatowej listy na string

uniq_vales_sentence = ' '.join(uniq_vales)

In [38]:
# tokenizacja stringu unikatowych wartości z kolumny tags

doc = nlp(uniq_vales_sentence)

In [39]:
# lemantyzacja wcześniej stworzonych tokenów. Stworzenie listy i listy tupli. Lista tupli zawierające słowo i jej podstawę słowotwórczą, sama lista zawiera wszystkie podstwy słowotwórcze 

all_lem = []
all_words = []
for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

In [40]:
# import pickle
# with open('all_lem.pkl', 'wb') as file:
#     pickle.dump(all_lem, file)

In [41]:
# Stworzenie słownika gdzie klucz to podstawa słowotwórcza a values jest listą słów dla tej podstawy występująca w kolumnie tags

lem_dict_0 = {}
for i in set(all_lem):
    lem_dict_0[i] = find_all_values_to_dictonary(i,all_words)

In [62]:
len(lem_dict_0['nieanulowanie'])

KeyError: 'nieanulowanie'

In [ ]:
# stworzenie słownika ze specjalnego obieku counter

uniq_values_count_dict = dict(uniq_values_count)

In [ ]:
# Stworzenie słownika, gdzie klucz to podstawa słwowtwórcza, a values to suma wartości występowania słów zawiązanych z tą lemą

number_of_occurence = {}
for i in lem_dict_0:
    number_of_occurence_value = []
    for j in lem_dict_0[i]:
        number_of_occurence_value.append(uniq_values_count_dict[j])
    number_of_occurence[i] = sum(number_of_occurence_value)

In [ ]:
# Posortowanie słownika zawierającego podstawę słowotwórczą oraz ilość wystąpnień słów z nim powiązanych

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [ ]:
df_number_of_occurence_sorted = pd.DataFrame(number_of_occurence_sorted, columns=['lema', 'ilość wystapień'])

In [ ]:
df_number_of_occurence_sorted.head(5)

,lema,ilość wystapień
0,zwrot,6127
1,reklamacja,4539
2,dostawa,3243
3,niedostarczona,2812
4,anulować,2808


In [ ]:
df_number_of_occurence_sorted.to_csv('number_of_occurence_sorted.csv', sep=',', encoding='cp1250',index=False)

In [43]:
labels_df = pd.read_csv('labels2.csv', sep=';', header=None)
labels_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,anulacja,anulować,anulacja,anulowac,anulacj,anulownie,anulownaie,nieanulowanie,anulaj,anuloewanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reklamacja,reklamacja,reklamation,reklamcja,reklaamcja,reklamajci,reklmacja,reklacja,reklama,reklamationa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wysyłka,wysyłce,wysyłka,wysyłke,wysyp,wysylce,wysyłkąć,wysyka,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refund,refund,refundacja,refunda,refunf,refung,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,przesyłka,przesyłka,przesylka,przesyłewka,przesyłkami,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
labels_df = labels_df.T
labels_df.columns = labels_df.iloc[0]
labels_df = labels_df[1:]

In [61]:
labels_df.head()

,anulacja,reklamacja,wysyłka,refund,przesyłka,dostawa,opóźnienie,uszkodzony,adres,ponowna,paczka,zapytanie,zwrot,brak,niekompletne,faktura,inne
1,anulować,reklamacja,wysyłce,refund,przesyłka,dostawa,opoznienie,uszkodzić,adres,ponowny,paczka,zapytanie,zwrot,brak,niekompletny,faktura,inny
2,anulacja,reklamation,wysyłka,refundacja,przesylka,niedostarczona,opoznić,uszkodzeć,adresse,NaN,paczkomacie,pytanie,zwrotny,brakować,niekomletny,fakura,NaN
3,anulowac,reklamcja,wysyłke,refunda,przesyłewka,niedostarczć,opoznieć,uszkodzenie,adresowy,NaN,paczkomat,zapytanienie,zwrtot,wybrakować,niekopletny,NaN,NaN
4,anulacj,reklaamcja,wysyp,refunf,przesyłkami,dostarczyć,opóźniić,uszkodozny,adresskl,NaN,paczkmać,zapytamie,zwrto,brakuć,niekomplet,NaN,NaN
5,anulownie,reklamajci,wysylce,refung,NaN,niedostarczono,opóźnienie,uszkopdzeć,zaadresować,NaN,paczk,zpytać,zwrócić,NaN,niekoempletne,NaN,NaN


In [46]:
lem_dict_1 = labels_df.to_dict(orient='list')

In [47]:
print(lem_dict_0['niedostarczić'])

{'niedostarczony'}


In [48]:
test1 = {'test1' : "['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",}

In [49]:
test1
key_lvl1_test = []
key_lvl1_test.append(test1.values())
print([print(i) for i in key_lvl1_test])

dict_values(["['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"])
[None]


In [50]:
from itertools import chain

def get_all_values (dict1: dict, dict0: dict) -> dict:
    final_dict = {}
    key_lvl1 = []
    key_lvl1.append(list(dict1.keys()))
    key_lvl1 = list(chain(*key_lvl1))
    for i in key_lvl1:
        temp_vals = []
        temp_keys = dict1[i]
        temp_keys = [x for x in temp_keys if x is not np.nan]
        for j in temp_keys:
            temp_vals.append(list(dict0[j]))
            print(temp_vals)
        temp_vals = set(item for sublist in temp_vals for item in sublist)
        final_dict[i] = temp_vals
        print(final_dict)
    return final_dict

In [51]:
get_all_values(lem_dict_1, lem_dict_0)

[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania']]
[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania'], ['anulację', 'anulacje', 'anulacja', 'anulacji']]
[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania'], ['anulację', 'anulacje', 'anulacja', 'anulacji'], ['anulowac']]
[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania'], ['anulację', 'anulacje', 'anulacja', 'anulacji'], ['anulowac'], ['anulacj']]
[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania'], ['anulację', 'anulacje', 'anulacja', 'anulacji'], ['anulowac'], ['anulacj'], ['anulownie']]
[['anulowano', 'anulowanie', 'anulowana', 'anulowane', 'anulować', 'nieanulowane', 'anulowani', 'anulowania'], ['anulację', 'anulacje', 'anulacja', 'anulacji'], [

KeyError: 'nieanulowanie'